In [9]:
import csv
import datetime as dt

# need to rewrite column headers to be simpler
with open('practice5.csv', 'r') as file, open('practice3.csv', 'w', newline='') as new:
    cols = ['Date', 'Weekday', '(GeV/Pass)', 'Operation Status', 'Hall A Exp', 'Hall A Params', 'Hall B Exp', 'Hall B Params', 
           'Hall C Exp', 'Hall C Params', 'Hall D Exp', 'Hall D Params', 'Priority', 'Pass', 'Notes']
    csv_writer = csv.writer(new)
    csv_reader = list(csv.reader(file))
    csv_reader.pop(0)
    csv_reader.insert(0, cols)
    for line in csv_reader:
        csv_writer.writerow(line)
    

with open('practice3.csv', 'r') as practice, open('Schedule.csv', 'w', newline="") as csv_file:
    prac = list(csv.DictReader(practice))
    sched = csv.DictWriter(csv_file, fieldnames=prac[0].keys(), delimiter=',')
    
    
    def format_date(d):
        """ Converts the date from mm/dd/yyyy format to yyyy-mm-dd format 
        (used by the datetime module) """
        # We need to format the dates because inputting mm/dd/yyyy can have some mistakes- not always perfect
        assert len(d) > 0    # Make sure there is a date, or something given
        month, day, year = d.split('/')
        d = dt.date(int(year), int(month), int(day))
        return d
    
    
    def revert_date(d):
        """ Converts the date from yyyy-mm-dd format to mm/dd/yyyy format 
        (used by the spreadsheet) """
        assert not d == 0    # Make sure there is a date, or something given
        d = str(d)
        year, month, day = d.split('-')
        d = month + '/' + day + '/' + year
        return d
        

    # pop out any lines without data
    popped = []
    for i in range(0, len(prac)-1):
        if i >= len(prac):
            break
        def p():
            if len(prac[i]['Date']) == 0:
                popped.append({'i' : i, 'data' : prac.pop(i)})
                p()
        p()
            
    
    def find_date(date):
        """ Binary search for the index of the date in the list of dates """
        first = 0
        last = len(prac)-1
        found = False

        while first <= last and not found:
            pos = 0
            midpoint = (first + last)//2
            if prac[midpoint]['Date'] == date:
                pos = midpoint
                found = True
            else:
                if date < prac[midpoint]['Date']:
                    last = midpoint-1
                else:
                    first = midpoint+1
        return pos, found
        
 
    def get_params(hall):
        """This program is for making changes to the accelerator's schedule."""
        print("You are making changes to the schedule for Hall " + hall + "\'s schedule")

        affirmative = ['y', 'yes', 'ye', 'yep', 'yup', '1', 'yeah', 'yea', 'correct', 'ya', 'yah', 'yeh']

        energy = float(input("To what energy will Hall " + hall + " be set? (GeV)"))

        if hall in ['b', 'B', 'd', 'D', '2', '4', 'two', 'four']:
            current = float(input("What current to Hall " + hall + "? (nanoAmps)"))
        else:
            current = float(input("What current to Hall " + hall + "? (microAmps)"))

        plr = input("Will the beam be polarized? (y/n)")
        freq = int(input("What frequency will the beam run at? (MHz)"))

        def ispolarized(plr):
            if plr.lower() in affirmative:
                return True
            else:
                return False

        start = str(input("What is the START date for this change? (mm/dd/yyyy)"))
        end = str(input("What is the END date for this change? (mm/dd/yyyy)"))

        expName = str(input('What is the name of the experiment the change is being made for?'))

        print('\nSo far, Hall ' + hall + '\'s Beam will be at: \n' 
          + str(energy) + ' GeV\n'
          + str(current) + ' microAmps\n'
          + 'Polarized: ' + str(ispolarized(plr)) + '\n'
          + str(freq) + ' MHz'
          + '\nFrom ' + start + ' to ' + end
          + '\nExperiment Name: ' + expName)

        assert str(input("Is this correct?")) in affirmative
        
        def format_params(energy, current, plr, freq):
            if plr:
                plr = 'p'
            else:
                plr = '-'
            return str(energy) + '/' + str(current) + '/' + plr + '/' + str(freq)
        
        return expName, format_date(start), format_date(end), format_params(energy, current, plr, freq)
    
    def write():
        # Need to convert dates back into normal form (mm/dd/yyyy)
        sched.writeheader()
        for line in prac:
            line['Date'] = revert_date(line['Date'])
            
        # Re-insert the popped columns:
        for d in popped:
            prac.insert(d['i'], d['data'])
        
        for line in prac:
            sched.writerow(line)
        
    def main():
        # Format dates to accomodate datetime module
        for line in prac:
            line['Date'] = format_date(line['Date'])
        
        hall = str(input("What hall's schedule to you want to edit? (A, B, C, or D)"))
        hall.upper()
        hall_params = 'Hall ' + hall + ' Params'
        hall_exp = 'Hall ' + hall + ' Exp'
        print(hall_params, hall_exp)
        
        name, start_day, end_day, params = get_params(hall)
        
        start_index, b = find_date(start_day)
        if not b:
            print("Date not found in file.")
        end_index, b = find_date(end_day)
        if not b:
            print("Date not found in file.")
            
        for i in range(start_index, end_index + 1):
            prac[i][hall_exp] = name
            prac[i][hall_params] = params
        
        write()
      
    main()



What hall's schedule to you want to edit? (A, B, C, or D) B


Hall B Params Hall B Exp
You are making changes to the schedule for Hall B's schedule


To what energy will Hall B be set? (GeV) 10.5
What current to Hall B? (nanoAmps) 200
Will the beam be polarized? (y/n) y
What frequency will the beam run at? (MHz) 250
What is the START date for this change? (mm/dd/yyyy) 6/14/2022
What is the END date for this change? (mm/dd/yyyy) 8/25/2022
What is the name of the experiment the change is being made for? RUN GROUP C/FT_ON



So far, Hall B's Beam will be at: 
10.5 GeV
200.0 microAmps
Polarized: True
250 MHz
From 6/14/2022 to 8/25/2022
Experiment Name: RUN GROUP C/FT_ON


Is this correct? y
